In [2]:
import os
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver #Webdriver de Selenium qui permet de contrôler un navigateur
from selenium.webdriver.common.by import By #Permet d'accéder aux différents élements de la page web
from webdriver_manager.chrome import ChromeDriverManager #Assure la gestion du webdriver de Chrome
from datetime import datetime, timedelta
import math
import time
import warnings
warnings.filterwarnings("ignore")
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import itertools

SELENIUM

In [3]:
options = Options()
options.headless = True
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--window-size=1920,1200")
options.add_argument("--download.default_directory=E:\\Corpus\\CLAPI_scrape")

In [4]:
driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)  #Initialisation du driver
driver.implicitly_wait(0.5)
time.sleep(3) #Ajout d'un temps de deux secondes avant de lancer l'action suivante
driver.get("http://clapi.icar.cnrs.fr/V3_Feuilleter.php?")

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:00<00:00, 11.8MB/s]


In [5]:
def next_corpus(driver):
    corpus_suivant = driver.find_element(By.XPATH, "//input[@value='Corpus suivant']")
    corpus_suivant.click()
    return driver

In [6]:
def get_corpus_name(driver):
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,features='html.parser')
    pattern = r'Corpus [0-9]+ sur 69 : (.*)' # Define the pattern using a regular expression
    tag = soup.find(text=re.compile(pattern)) # Find the tag that contains the pattern
    match = re.search(pattern, tag) # Extract the matching text using the regular expression pattern
    corpus_name = match.group(1) # Extract the text group and save it in the corpus_name variable
    return corpus_name # Print the corpus_name variable

In [7]:
def get_metadata_corpus(page_html, driver):
    soup = BeautifulSoup(page_html,features='html.parser')
    pattern = r'Corpus [0-9]+ sur 69 : (.*)' 
    tag = soup.find(text=re.compile(pattern))
    match = re.search(pattern, tag) 
    corpus_name = match.group(1) 
    metadata_corpus = soup.find('div', {'id':'u262container'})
    autrice_corpus = metadata_corpus.find('div', {'id':'u274_rtf'}).get_text()
    date_collecte_corpus = metadata_corpus.find('div', {'id':'u276_rtf'}).get_text()
    duree_corpus = metadata_corpus.find('div', {'id':'u280_rtf'}).get_text()
    langue_corpus = metadata_corpus.find('div', {'id':'u282_rtf'}).get_text()
    print('Corpus_name : ', corpus_name)
    print('\n')
    print('auteur.rice : ', autrice_corpus)
    print('date_collecte_corpus : ', date_collecte_corpus)
    print('duree_corpus : ', duree_corpus)
    print('langue_corpus : ', langue_corpus)
    nb_enregistrement, liste_df = enregistrement(soup, driver)
    print('nb_enregistrement: ', nb_enregistrement)
    for df in liste_df:
        df['nom_corpus'] = corpus_name
        df['responsable'] = autrice_corpus
        df['duree_corpus'] = duree_corpus
        df['langue_corpus'] = langue_corpus
    print('___________________________________')
    return liste_df

In [8]:
def get_element(liste):
    if not liste:
        liste = ''
    elif len(liste)>1:
        liste = liste[-1].get_text()
    elif len(liste) == 1:
        liste = liste[0].get_text()
    else : 
        liste = ''    
    return liste

In [9]:
def enregistrement(soup, driver):
    nb_enregistrements = int(soup.find('div', {'id':'u286_rtf'}).get_text().replace(' Enregistrements ', '').replace(' Enregistrement ', ''))
    liste_enregistrement = soup.find_all('div', {'id':'ou344'})
    liste_df = []
    for i in range(nb_enregistrements):
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,features='html.parser')
        nom_enregistrement = soup.find('div', {'id':'u524_rtf'}).get_text()
    
        description = soup.find_all('div', {'id':'u380'})
        description = get_element(description)
        
        data_collecte = soup.find_all('div', {'id':'u372_rtf'})
        data_collecte = get_element(data_collecte)

        duree = soup.find_all('div', {'id':'u374_rtf'})
        duree = get_element(duree)

        nb_locuteurs = soup.find_all('div', {'id':'u376_rtf'})
        nb_locuteurs = get_element(nb_locuteurs)
        
        langue_lieu = soup.find_all('div', {'id':'u378_rtf'})
        langue_lieu = get_element(langue_lieu)

        table_locuteur = soup.find_all('div', {'id':'u392container'})[-1]
        noms_locuteurs = [x.get_text() for x in table_locuteur.find_all('div', {'id':'u404_rtf'})]
        sexe_locuteurs = [x.get_text() for x in table_locuteur.find_all('div', {'id':'u406_rtf'})]
        age_locuteurs = [x.get_text() for x in table_locuteur.find_all('div', {'id':'u408_rtf'})]
        langue_locuteurs = [x.get_text() for x in table_locuteur.find_all('div', {'id':'u410_rtf'})]
        profession_locuteurs = [x.get_text() for x in table_locuteur.find_all('div', {'id':'u412_rtf'})]
        print('\n')
        print('ENREGISTREMENT {}'.format(i+1))
        print('nom_enregistrement : ', nom_enregistrement)
        print('description : ', description)
        print('data_collecte : ', data_collecte)
        print('duree : ', duree)
        print('nb_locuteurs : ', nb_locuteurs)
        print('langue_lieu : ', langue_lieu)
        print('noms_locuteurs : ', noms_locuteurs)
        print('sexe_locuteurs : ', sexe_locuteurs)
        print('age_locuteurs : ', age_locuteurs)
        print('langue_locuteurs : ', langue_locuteurs)
        print('profession_locuteurs : ', profession_locuteurs)
        dict_loc = {}
        for index in range(len(noms_locuteurs)):
            dict_loc[index] = {
        'nom': noms_locuteurs[index],
        'sexe': sexe_locuteurs[index],
        'age': age_locuteurs[index],
        'profession': profession_locuteurs[index],
        'langue': langue_locuteurs[index]}

        # Find the element using its XPath
        element = driver.find_element(By.XPATH, xpath_soup(liste_enregistrement[i]))
        driver.execute_script("arguments[0].click();", element)
        time.sleep(2)

        df = pd.DataFrame({'nom_enregistrement': nom_enregistrement,
            'description': description,
            'date_collecte': data_collecte,
            'duree': duree,
            'nb_locuteurs': nb_locuteurs,
            'langue_lieu': langue_lieu,
            'locuteurs': str(dict_loc) }, index=[0])

        liste_df.append(df)
    return nb_enregistrements, liste_df

In [10]:
def xpath_soup(element):
    components = []
    child = element if element.name else element.parent
    for parent in child.parents:
        siblings = parent.find_all(child.name, recursive=False)
        components.append(
            child.name
            if siblings == [child] else
            '%s[%d]' % (child.name, 1 + siblings.index(child))
            )
        child = parent
    components.reverse()
    return '/%s' % '/'.join(components)

In [11]:
liste_data_clapi = []
nom_ancien = ''
while nom_ancien != get_corpus_name(driver):
    print('nom ancien: ', nom_ancien)
    print('nom actuel: ', get_corpus_name(driver))
    liste_df = get_metadata_corpus(driver.page_source, driver)
    liste_data_clapi.append(liste_df)
    time.sleep(1)
    nom_ancien = get_corpus_name(driver)
    time.sleep(1)
    next_corpus(driver)

nom ancien:  
nom actuel:  Apéritif entre ami(e)s - chat
Corpus_name :  Apéritif entre ami(e)s - chat


auteur.rice :  G. Icor, S. SchwarzConseil de gestion Clapi
date_collecte_corpus :  21 Nov 2009
duree_corpus :  31m55s
langue_corpus :  Français


ENREGISTREMENT 1
nom_enregistrement :  Apéritif entre ami(e)s - chat
description :  Interaction entre quatre amis durant un apéritif.
data_collecte :  21 Nov 2009
duree :  31m55s
nb_locuteurs :  4
langue_lieu :  françaisFRANCE  Région Lyonnaise
noms_locuteurs :  ['JUL Julie', 'CLA Claire', 'LAU Laurent', 'JEA Jean']
sexe_locuteurs :  ['Femme', 'Femme', 'Homme', 'Homme']
age_locuteurs :  ['20 - 30', '20 - 30', '20 - 30', '20 - 30']
langue_locuteurs :  ['', 'français', 'français', 'français']
profession_locuteurs :  ['en scolarité', '', 'en activité', 'en scolarité']
nb_enregistrement:  1
___________________________________
nom ancien:  Apéritif entre ami(e)s - chat
nom actuel:  Apéritif entre ami(e)s - glasgow
Corpus_name :  Apéritif entre a



ENREGISTREMENT 9
nom_enregistrement :  Inscription à l’université Lyon 2
description :  Plusieurs étudiantes allemandes qui commencent un semestre d'études à Lyon, se trouvent au secrétariat pour régler des problèmes d’inscription.

data_collecte :  Oct 1984
duree :  6m57s
nb_locuteurs :  5
langue_lieu :  français 
noms_locuteurs :  ['G', 'Doris', 'Andrea', 'Isadora']
sexe_locuteurs :  ['Homme', 'Femme', 'Femme', 'Femme']
age_locuteurs :  ['30 - 50', '24', '31', '23']
langue_locuteurs :  ['français', 'allemand', 'allemand', 'allemand']
profession_locuteurs :  ['', '', '', '']
nb_enregistrement:  9
___________________________________
nom ancien:  Bielefeld _ situations de contact – cadre universitaire
nom actuel:  Bielefeld _ situations de contact – conversations privées
Corpus_name :  Bielefeld _ situations de contact – conversations privées


auteur.rice :  U. Krafft, E. Gülich, U. Dausendschön-GayConseil de gestion Clapi, U. Krafft, E. Gülich, U. Dausendschön-Gay
date_collecte_corp



ENREGISTREMENT 16
nom_enregistrement :  Visite chez l’ancienne logeuse II
description :  Une étudiante allemande rend visite à son ancienne propriétaire française, conversation.
data_collecte :  Mai 1985
duree :  21m3s
nb_locuteurs :  2
langue_lieu :  français 
noms_locuteurs :  ['Mme Mougin', 'Susanne']
sexe_locuteurs :  ['Femme', 'Femme']
age_locuteurs :  ['60 - 63', '23']
langue_locuteurs :  ['français', 'allemand']
profession_locuteurs :  ['', '']


ENREGISTREMENT 17
nom_enregistrement :  Visite chez l’ancienne logeuse II
description :  Une étudiante allemande rend visite à son ancienne propriétaire française, conversation.
data_collecte :  Mai 1985
duree :  21m3s
nb_locuteurs :  2
langue_lieu :  français 
noms_locuteurs :  ['Mme Mougin', 'Susanne']
sexe_locuteurs :  ['Femme', 'Femme']
age_locuteurs :  ['60 - 63', '23']
langue_locuteurs :  ['français', 'allemand']
profession_locuteurs :  ['', '']


ENREGISTREMENT 18
nom_enregistrement :  Visite chez l’ancienne logeuse II
descript



ENREGISTREMENT 8
nom_enregistrement :  Conférence sur l`eau
description :  Conférence sur l'eau à laquelle assistent les jeunes européens : une intervenante et un traducteur anglophone.
data_collecte :  Oct 2008
duree :  13m8s
nb_locuteurs :  2
langue_lieu :  français,anglais 
noms_locuteurs :  ['ger', 'ann']
sexe_locuteurs :  ['Homme', 'Homme']
age_locuteurs :  ['43', '30 - 60']
langue_locuteurs :  ['anglais', 'français']
profession_locuteurs :  ['en activité', 'inconnue']


ENREGISTREMENT 9
nom_enregistrement :  Conférence sur l`eau
description :  Conférence sur l'eau à laquelle assistent les jeunes européens : une intervenante et un traducteur anglophone.
data_collecte :  Oct 2008
duree :  13m8s
nb_locuteurs :  2
langue_lieu :  français,anglais 
noms_locuteurs :  ['ger', 'ann']
sexe_locuteurs :  ['Homme', 'Homme']
age_locuteurs :  ['43', '30 - 60']
langue_locuteurs :  ['anglais', 'français']
profession_locuteurs :  ['en activité', 'inconnue']


ENREGISTREMENT 10
nom_enregistrement

nom ancien:  Rédaction conversationnelle - thème: la disparition des petits commerces
nom actuel:  Rédaction conversationnelle - thème: les devoirs à la maison
Corpus_name :  Rédaction conversationnelle - thème: les devoirs à la maison


auteur.rice :  M. De Gaulmyn, EtudiantConseil de gestion Clapi
date_collecte_corpus :  1995
duree_corpus :  4h40m
langue_corpus :  Français


ENREGISTREMENT 1
nom_enregistrement :  Edyta maria 1
description :  Co-rédaction d'un texte argumentatif par des étudiants étrangers sur le thème des devoirs à la maison.
data_collecte :  1995
duree :  1h16m
nb_locuteurs :  2
langue_lieu :  français 
noms_locuteurs :  ['e', 'm']
sexe_locuteurs :  ['Femme', 'Femme']
age_locuteurs :  ['18 - 30', '18 - 30']
langue_locuteurs :  ['polonais', 'portugais']
profession_locuteurs :  ['en scolarité', 'en scolarité']


ENREGISTREMENT 2
nom_enregistrement :  Edyta maria 1
description :  Co-rédaction d'un texte argumentatif par des étudiants étrangers sur le thème des devoirs 



ENREGISTREMENT 1
nom_enregistrement :  Mosaic - architecture
description :  Enregistrement vidéo d'une réunion de conception entre trois architectes (et une observatrice) à qui la réhabilitation d'un château en centre de séminaires a été confiée.
data_collecte :  Nov 2002
duree :  1h18m45s
nb_locuteurs :  3
langue_lieu :  françaisFRANCE  
noms_locuteurs :  ['c', 'm', 'l']
sexe_locuteurs :  ['Homme', 'Femme', 'Homme']
age_locuteurs :  ['40 - 50', '30 - 40', '30 - 40']
langue_locuteurs :  ['français', '', 'français']
profession_locuteurs :  ['en activité, architecte', "en activité, architecte d'intérieur", "en activité, architecte,architecte d'intérieur"]
nb_enregistrement:  1
___________________________________
nom ancien:  Réunion de conception en architecture - mosaic
nom actuel:  Réunion de préparation de la fête des conscrits
Corpus_name :  Réunion de préparation de la fête des conscrits


auteur.rice :  G. Icor
date_collecte_corpus :  2008
duree_corpus :  1h41m
langue_corpus :  F



ENREGISTREMENT 8
nom_enregistrement :  10- julien 10.08
description :  
data_collecte :  
duree :  2m52s
nb_locuteurs :  2(1 décrits)
langue_lieu :  français 
noms_locuteurs :  ['chi']
sexe_locuteurs :  ['Homme']
age_locuteurs :  ['10,08']
langue_locuteurs :  ['']
profession_locuteurs :  ['inconnue']


ENREGISTREMENT 9
nom_enregistrement :  5- lucie 5.10
description :  
data_collecte :  
duree :  10m34s
nb_locuteurs :  2(1 décrits)
langue_lieu :  français 
noms_locuteurs :  ['chi']
sexe_locuteurs :  ['Femme']
age_locuteurs :  ['5,10']
langue_locuteurs :  ['']
profession_locuteurs :  ['inconnue']


ENREGISTREMENT 10
nom_enregistrement :  5- jean-baptiste 5.05
description :  
data_collecte :  
duree :  9m39s
nb_locuteurs :  2(1 décrits)
langue_lieu :  français 
noms_locuteurs :  ['chi']
sexe_locuteurs :  ['Homme']
age_locuteurs :  ['5,05']
langue_locuteurs :  ['']
profession_locuteurs :  ['inconnue']


ENREGISTREMENT 11
nom_enregistrement :  5- marion 5.06
description :  
data_collecte



ENREGISTREMENT 36
nom_enregistrement :  7- p 7.04 u
description :  
data_collecte :  
duree :  4m54s
nb_locuteurs :  2(1 décrits)
langue_lieu :  français 
noms_locuteurs :  ['chi']
sexe_locuteurs :  ['']
age_locuteurs :  ['7,04']
langue_locuteurs :  ['']
profession_locuteurs :  ['inconnue']


ENREGISTREMENT 37
nom_enregistrement :  7- p 7.04 z
description :  
data_collecte :  
duree :  3m46s
nb_locuteurs :  2(1 décrits)
langue_lieu :  français 
noms_locuteurs :  ['chi']
sexe_locuteurs :  ['Femme']
age_locuteurs :  ['7,04']
langue_locuteurs :  ['']
profession_locuteurs :  ['inconnue']


ENREGISTREMENT 38
nom_enregistrement :  7- p 7.08 b
description :  
data_collecte :  
duree :  6m45s
nb_locuteurs :  2(1 décrits)
langue_lieu :  français 
noms_locuteurs :  ['chi']
sexe_locuteurs :  ['Homme']
age_locuteurs :  ['7,08']
langue_locuteurs :  ['']
profession_locuteurs :  ['inconnue']


ENREGISTREMENT 39
nom_enregistrement :  7- p 7.08 o
description :  
data_collecte :  
duree :  7m11s
nb_lo



ENREGISTREMENT 64
nom_enregistrement :  7- p 7.06 bb
description :  
data_collecte :  
duree :  6m37s
nb_locuteurs :  2(1 décrits)
langue_lieu :  français 
noms_locuteurs :  ['chi']
sexe_locuteurs :  ['Femme']
age_locuteurs :  ['7,06']
langue_locuteurs :  ['']
profession_locuteurs :  ['inconnue']


ENREGISTREMENT 65
nom_enregistrement :  7- p 7.09 o
description :  
data_collecte :  
duree :  4m9s
nb_locuteurs :  2(1 décrits)
langue_lieu :  français 
noms_locuteurs :  ['chi']
sexe_locuteurs :  ['Femme']
age_locuteurs :  ['7,09']
langue_locuteurs :  ['']
profession_locuteurs :  ['inconnue']


ENREGISTREMENT 66
nom_enregistrement :  7- p 7.10 l
description :  
data_collecte :  
duree :  4m17s
nb_locuteurs :  2(1 décrits)
langue_lieu :  français 
noms_locuteurs :  ['chi']
sexe_locuteurs :  ['Femme']
age_locuteurs :  ['7,10']
langue_locuteurs :  ['']
profession_locuteurs :  ['inconnue']


ENREGISTREMENT 67
nom_enregistrement :  7- raphaël 7.06
description :  
data_collecte :  
duree :  5m



ENREGISTREMENT 92
nom_enregistrement :  P 20 a
description :  
data_collecte :  
duree :  2m41s
nb_locuteurs :  2(1 décrits)
langue_lieu :  français 
noms_locuteurs :  ['chi']
sexe_locuteurs :  ['']
age_locuteurs :  ['20']
langue_locuteurs :  ['']
profession_locuteurs :  ['inconnue']


ENREGISTREMENT 93
nom_enregistrement :  P 20 a
description :  
data_collecte :  
duree :  2m41s
nb_locuteurs :  2(1 décrits)
langue_lieu :  français 
noms_locuteurs :  ['chi']
sexe_locuteurs :  ['']
age_locuteurs :  ['20']
langue_locuteurs :  ['']
profession_locuteurs :  ['inconnue']


ENREGISTREMENT 94
nom_enregistrement :  P 20 a
description :  
data_collecte :  
duree :  2m41s
nb_locuteurs :  2(1 décrits)
langue_lieu :  français 
noms_locuteurs :  ['chi']
sexe_locuteurs :  ['']
age_locuteurs :  ['20']
langue_locuteurs :  ['']
profession_locuteurs :  ['inconnue']


ENREGISTREMENT 95
nom_enregistrement :  P 20 a
description :  
data_collecte :  
duree :  2m41s
nb_locuteurs :  2(1 décrits)
langue_lieu 

In [12]:
df= liste_data_clapi[3][0]

In [13]:
df

,nom_enregistrement,description,date_collecte,duree,nb_locuteurs,langue_lieu,locuteurs,nom_corpus,responsable,duree_corpus,langue_corpus
0,Apéritif entre ami(e)s - rupture,Interaction entre trois amis pendant un apéritif.,8 Nov 2009,33m,3,françaisFRANCE Région Lyonnaise,"{0: {'nom': 'ALB Albine', 'sexe': 'Femme', 'ag...",Apéritif entre ami(e)s - rupture,"G. Icor, J. DuchanoisConseil de gestion Clapi",33m,Français


In [14]:
driver.quit()